## Launch the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [1]:
import numpy as np
from pathlib import Path
import subprocess

#### Create an executable file of STEMMUS_SCOPE

In [3]:
# path to executable
root_path_to_model = Path(Path.home(),"EcoExtreML", "STEMMUS_SCOPE")
# submit slurm job
result = subprocess.run(["sbatch", Path(root_path_to_model, "exe", "build_stemmus_scope_exe.sh")])
result.check_returncode()

success!


#### Update/set config files

In [8]:
# the user must provide the correct path
path_dict = {
    "SoilPropertyPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/SoilProperty/",
    "InputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/",
    "OutputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/output/",
    "ForcingPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/",
    "ForcingFileName": "AU-DaS_2010-2017_OzFlux_Met.nc",
    "DurationSize": "NA"
}

# generate a text file
root_path = Path().resolve().parent

with open(Path(root_path, "config_file_snellius.txt"), 'w') as f:
    print("The listed paths are defined in the config file:")
    for i in path_dict.keys():
        print(i)
        f.write((i + "=" + path_dict[i] + "\n"))

The listed paths are defined in the config file:
SoilPropertyPath
InputPath
OutputPath
ForcingPath
ForcingFileName
DurationSize
